In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cash-loans/prev_approved_cash_loans.csv
/kaggle/input/cash-loans/curr_cash_loans.csv
/kaggle/input/raw-data/POS_CASH_balance.csv
/kaggle/input/raw-data/installments_payments.csv
/kaggle/input/raw-data/previous_application-home-credit.csv


In [2]:
from pathlib import Path
raw_dir = Path('/kaggle/input/raw-data')
cash_loan_dir = Path('/kaggle/input/cash-loans')
output_dir = Path('/kaggle/working')

instal_csv = 'installments_payments.csv'
curr_cash_csv = 'curr_cash_loans.csv'
prev_cash_csv = 'prev_approved_cash_loans.csv'
pos_csv = 'POS_CASH_balance.csv'

In [91]:
prev_cash_df = pd.read_csv(cash_loan_dir / prev_cash_csv)
curr_cash_df = pd.read_csv(cash_loan_dir / curr_cash_csv)
pos_df = pd.read_csv(raw_dir / pos_csv)
instal_df = pd.read_csv(raw_dir / instal_csv)

In [92]:
## sort instal_df for easier visual inspection according to installment number.
instal_df_sorted = instal_df.sort_values(
    by=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER', 'NUM_INSTALMENT_VERSION'])

### Duplicate Examples
Use 3 cases below to explain the causes of duplicates.

### Example 1
Consider **SK_ID_PREV == 12_393_48** and **NUM_INSTALMENT_NUMBER == 23**. <br>
The same instalment number has 2 versions: 1 and 2. <br>
Note that **AMT_INSTALMENT** == **AMT_PAYMENT** for **NUM_INSTALMENT_VERSION == 1/2** and **NUM_INSTALMENT_NUMBER == 23** on **DAYS_ENTRY_PAYMENT == -1196**.

In [93]:
filter_cond = (instal_df_sorted.SK_ID_PREV == 12_393_48) \
                & (instal_df_sorted.NUM_INSTALMENT_NUMBER == 23)
instal_df_sorted[filter_cond]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
642124,1239348,136896,1.0,23,-1196.0,-1196.0,12979.08,12979.08
1967766,1239348,136896,1.0,23,-1196.0,-1166.0,12979.08,5637.33
2815271,1239348,136896,1.0,23,-1196.0,-1196.0,12979.08,856.98
2869838,1239348,136896,1.0,23,-1196.0,-1174.0,12979.08,5850.00
642125,1239348,136896,2.0,23,-1196.0,-1196.0,12344.31,12979.08
1967767,1239348,136896,2.0,23,-1196.0,-1166.0,12344.31,5637.33
2815272,1239348,136896,2.0,23,-1196.0,-1196.0,12344.31,856.98
2869839,1239348,136896,2.0,23,-1196.0,-1174.0,12344.31,5850.00


### Example 2
Consider **SK_ID_PREV == 28_434_84** and **NUM_INSTALMENT_NUMBER == 11, 12**. <br>
Each instalment number is splitted into 2 partial repayments. <br>
Note that **TOTAL_REPAID_PER_INSTAL == AMT_INSTALMENT**.

In [94]:
filter_cond = (instal_df_sorted.SK_ID_PREV == 28_434_84) \
                & (instal_df_sorted.NUM_INSTALMENT_NUMBER.isin([11, 12]))
instal_df_sorted[filter_cond]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
4248106,2843484,229590,1.0,11,-1168.0,-1197.0,5039.730,333.630
5991645,2843484,229590,1.0,11,-1168.0,-1162.0,5039.730,4706.100
4093862,2843484,229590,1.0,12,-1138.0,-1162.0,5036.355,333.900
7553708,2843484,229590,1.0,12,-1138.0,-1136.0,5036.355,4702.455


### Example 3
Consider **SK_ID_PREV == 26_313_84** and **NUM_INSTALMENT_NUMBER == 1**. <br>
Note duplicates occur when we consider the following columns:
1. **SK_ID_PREV**.
2. **NUM_INSTALMENT_NUMBER**.
3. **DAYS_ENTRY_PAYMENT**.
4. **AMT_PAYMENT**.

Realise that **AMT_PAYMENT** > **AMT_INSTALMENT** for version 1. <br>
Hence, a new row with version 2 instalment is created to balance the excess **AMT_PAYMENT** made in version 1. <br>
To confirm: (54_022.140 + 61_522_9.515) == 66_925_1.655.

In [95]:
filter_cond = (instal_df_sorted.SK_ID_PREV == 26_313_84) \
                & (instal_df_sorted.NUM_INSTALMENT_NUMBER == 1)
instal_df_sorted[filter_cond]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
11721916,2631384,456255,1.0,1,-756.0,-768.0,54022.140,669251.655
11721915,2631384,456255,2.0,1,-756.0,-768.0,615229.515,669251.655


### A step to remove duplicates.
1. Group by **SK_ID_PREV**, **NUM_INSTALMENT_NUMBER**, then get **MIN(NUM_INSTALMENT_VERSION)**, assign to min_version_df.
2. Inner join min_version_df and instal_df_sorted, this eliminates the issue related to Example 3.
3. Removing floating points for **AMT_INSTALMENT** and **AMT_PAYMENT**. Do a groupby to compute **TOTAL_REPAID_PER_INSTAL**.
4. Filter to according to the following:
    1. **AMT_INSTALMENT <= AMT_PAYMENT**: to account for exact payment or prepayment.
    2. **TOTAL_REPAID_PER_INSTAL == AMT_INSTALMENT**: to account for multiple partial payments for 1 instalment.
    3. This removes the issue related to Example 3.

In [96]:
min_version_df = instal_df_sorted.groupby(['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'])['NUM_INSTALMENT_VERSION'].min().reset_index()

In [119]:
instal_df_cleaned = instal_df_sorted.merge(min_version_df, how='inner', on=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER', 'NUM_INSTALMENT_VERSION'])

In [120]:
#Easier to work without floating points
instal_df_cleaned['AMT_INSTALMENT'] = instal_df_cleaned['AMT_INSTALMENT'].round()
instal_df_cleaned['AMT_PAYMENT'] = instal_df_cleaned['AMT_PAYMENT'].round()
instal_df_cleaned['TOTAL_REPAID_PER_INSTAL'] = instal_df_cleaned\
                                                .groupby(by=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'])['AMT_PAYMENT'].transform(np.sum)

In [121]:
filter_cond_1 = (instal_df_cleaned.AMT_INSTALMENT \
                     <= instal_df_cleaned.AMT_PAYMENT) #to account for prepayment

filter_cond_2 = (instal_df_cleaned.TOTAL_REPAID_PER_INSTAL \
                     == instal_df_cleaned.AMT_INSTALMENT)

In [122]:
instal_df_no_dup = instal_df_cleaned[(filter_cond_1) | (filter_cond_2)].copy()

## Check the 3 Cases again
Expectations:
1. For **SK_ID_PREV 12_393_48**, should see 1 row for **NUM_INSTALMENT_VERSION 1, NUM_INSTALMENT_NUMBER 23**.
2. For **SK_ID_PREV 28_434_84**, no change, the original rows are valid.
3. For **SK_ID_PREV 26_313_84**, should see 1 row for **NUM_INSTALMENT_VERSION 1, NUM_INSTALMENT_NUMBER 1**.

In [123]:
filter_cond = (instal_df_no_dup.SK_ID_PREV == 12_393_48) \
                & (instal_df_no_dup.NUM_INSTALMENT_NUMBER == 23)
instal_df_no_dup[filter_cond]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,TOTAL_REPAID_PER_INSTAL
1870473,1239348,136896,1.0,23,-1196.0,-1196.0,12979.0,12979.0,25323.0


In [124]:
filter_cond = (instal_df_no_dup.SK_ID_PREV == 28_434_84) \
                & (instal_df_no_dup.NUM_INSTALMENT_NUMBER.isin([11, 12]))
instal_df_no_dup[filter_cond]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,TOTAL_REPAID_PER_INSTAL
13515122,2843484,229590,1.0,11,-1168.0,-1197.0,5040.0,334.0,5040.0
13515123,2843484,229590,1.0,11,-1168.0,-1162.0,5040.0,4706.0,5040.0
13515124,2843484,229590,1.0,12,-1138.0,-1162.0,5036.0,334.0,5036.0
13515125,2843484,229590,1.0,12,-1138.0,-1136.0,5036.0,4702.0,5036.0


In [125]:
filter_cond = (instal_df_no_dup.SK_ID_PREV == 26_313_84) \
                & (instal_df_no_dup.NUM_INSTALMENT_NUMBER == 1)
instal_df_no_dup[filter_cond]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,TOTAL_REPAID_PER_INSTAL
12019598,2631384,456255,1.0,1,-756.0,-768.0,54022.0,669252.0,669252.0


### Preparing Data For Feature Extraction
1. Drop **TOTAL_REPAID_PER_INSTAL** column and recompute it again.
2. Drop **AMT_ANNUITY** and inner join with **prev_cash_df** on **SK_ID_PREV**.
3. Inner Join with **pos_df** on **SK_ID_PREV**.
4. Inner Join with **curr_cash_df** on **SK_ID_CURR**.


10. Drop the following columns as they will not be needed:
    1. **NUM_INSTALMENT_VERSION**.

In [ ]:
cleaned_instal_df.drop('TOTAL_REPAID_PER_INSTAL', axis=1, inplace=True)
cleaned_instal_df['TOTAL_REPAID_PER_INSTAL'] = cleaned_instal_df \
                                                .groupby(by=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'])['AMT_PAYMENT'].transform(np.sum)

In [ ]:
print(f'cols: {cleaned_instal_df.columns}')
print(cleaned_instal_df.shape)

In [ ]:
prev_cash_cols = ['SK_ID_PREV', 'NAME_PRODUCT_TYPE', 'NAME_YIELD_GROUP', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE']
pos_df_copy = pos_df.copy()
combined_pos_prev_cash = prev_cash_df[prev_cash_cols].merge(
    pos_df_copy, how='inner', on=['SK_ID_PREV'])

In [ ]:
cleaned_instal_df.drop('AMT_ANNUITY', axis=1, inplace=True)
joined_1 = cleaned_instal_df.merge(
        combined_pos_prev_cash, how='inner', on=['SK_ID_PREV'])

In [ ]:
joined_1.shape

In [ ]:
joined_1.columns

In [ ]:
new_names = {'AMT_CREDIT': 'CURR_AMT_CREDIT', 'AMT_ANNUITY': 'CURR_AMT_ANNUITY'}
curr_cash_df.rename(new_names, axis=1, inplace=True)

joined_2 = joined_1.merge(
    curr_cash_df, how='inner', on=['SK_ID_CURR'])

In [ ]:
joined_2.shape

In [ ]:
joined_2.columns

In [ ]:
joined_2[['SK_ID_CURR', 'TARGET']].drop_duplicates().groupby('TARGET')['TARGET'].count()

In [ ]:
joined_2.isnull().sum(axis=0)

## 4 Missing Data for DAYS_ENTRY_PAYMENT AND AMT_PAYMENT
**AMT_INSTALMENT == 0** causes the null data. <br>
Just remove **AMT_INSTALMENT == 0**.

In [ ]:
joined_2[joined_2.DAYS_ENTRY_PAYMENT.isnull()] #113_144_2, 138_428_3, 194_879_2, 202_886_5

In [ ]:
joined_2.dtypes

In [ ]:
joined_2 = joined_2[joined_2.AMT_INSTALMENT != 0]

In [ ]:
joined_2.isnull().sum(axis=0)

In [ ]:
joined_2.to_csv(output_dir / 'de_duplicated_data.csv')